In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from core import solve, solve_sample, prod, read_sample_input, read_input, xor

import toolz.curried as tz
import operator
import time
import collections

import re

from dataclasses import dataclass

# day 1

In [3]:
sample_1 = """1
1721
979
366
299
675
1456
1990"""

In [4]:
@tz.curry
def two_elems_sum_to_target(target, sequence, is_sorted=False):
    """
    Find two elements in a sequence that sum to the target
    """
    def recurse(target, seq):
        if len(seq) < 2:
            return None
        first, last = seq[0], seq[-1]
        if first + last == target:
            return first, last
        if first + last < target:
            return recurse(target, seq[1:])
        return recurse(target, seq[:-1])

    return recurse(target, sequence if is_sorted else sorted(sequence))

In [5]:
def solve_1a(inp):
    return tz.pipe(
        inp, 
        tz.map(int), 
        list, 
        two_elems_sum_to_target(2020), 
        prod)

In [6]:
solve_sample(solve_1a, sample_1), solve(solve_1a, 1)

(514579, 972576)

In [7]:
@tz.curry
def three_elems_sum_to_target(target, sequence):
    sorted_sequence = sorted(sequence)
    for idx, elem in enumerate(sorted_sequence):
        result = two_elems_sum_to_target(
            target - elem, 
            sorted_sequence[:idx] + sorted_sequence[(idx+1):], 
            is_sorted=True
        )
        if result:
            return elem, *result


In [8]:
def solve_1b(inp):
    return tz.pipe(
        inp, 
        tz.map(int), 
        list, 
        three_elems_sum_to_target(2020), 
        prod)

In [9]:
solve_sample(solve_1b, sample_1), solve(solve_1b, 1)

(241861950, 199300880)

# day 2

In [10]:
sample_2 = """1-3 a: abcde
1-3 b: cdefg
2-9 c: ccccccccc"""

In [11]:
password_pattern = re.compile("(\d+)-(\d+)\s(\w):\s(\w+)")

@dataclass
class PasswordInput:
    first: int
    second: int
    char: str
    password: str

def re_to_passwordinput(match):
    first, second, char, password = match[0]
    return PasswordInput(int(first), int(second), char, password)

def password_is_valid(password_input):
    return password_input.first <= password_input.password.count(password_input.char) <= password_input.second

In [12]:
def solve_2a(inp):
    return tz.pipe(
        inp,
        tz.map(password_pattern.findall),
        tz.map(re_to_passwordinput),
        tz.filter(password_is_valid),
        tz.count
    )

In [13]:
solve_sample(solve_2a, sample_2), solve(solve_2a, 2)

(2, 493)

In [14]:
def password_is_valid_b(password_input):
    first_valid = password_input.password[password_input.first - 1] == password_input.char
    second_valid = password_input.password[password_input.second - 1] == password_input.char
    return xor(first_valid, second_valid)


In [15]:
def solve_2b(inp):
    return tz.pipe(
        inp,
        tz.map(password_pattern.findall),
        tz.map(re_to_passwordinput),
        tz.filter(password_is_valid_b),
        tz.count
    )

In [16]:
solve_sample(solve_2b, sample_2), solve(solve_2b, 2)

(1, 593)

# day 3

In [17]:
sample_3 = """..##.......
#...#...#..
.#....#..#.
..#.#...#.#
.#...##..#.
..#.##.....
.#.#.#....#
.#........#
#.##...#...
#...##....#
.#..#...#.#"""

In [18]:
def rot_index(idx, arr):
    return idx % len(arr)

def rot_value_at(idx, arr):
    return arr[rot_index(idx, arr)]


In [19]:
@tz.curry
def count_trees_and_advance(right, state, section):
    location, trees = state
    trees += (rot_value_at(location, section) == '#')
    location += right
    return (location, trees)

In [20]:
@tz.curry
def toboggan_travel(down, right, forest):
    return tz.pipe(
        forest,
        tz.take_nth(down),
        lambda forest: tz.reduce(count_trees_and_advance(right), forest, (0, 0)),
        tz.last
    )

In [21]:
def solve_3a(inp):
    return tz.pipe(
        inp,
        tz.map(lambda s: s.strip()),
        toboggan_travel(1, 3)
    )

In [22]:
solve_sample(solve_3a, sample_3), solve(solve_3a, 3)

(7, 223)

In [23]:
def solve_3b(inp):
    forest = tz.pipe(
        inp,
        tz.map(lambda s: s.strip()),
        list
    )
    paths = [(1, 1), (1, 3), (1, 5), (1, 7), (2, 1)]
    return tz.pipe(
        paths,
        tz.map(lambda x: toboggan_travel(*x)(forest)),
        prod
    )

In [24]:
solve_sample(solve_3b, sample_3), solve(solve_3b, 3)

(336, 3517401300)

# day 4

In [25]:
sample_4 = """ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in"""

required_fields = ['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid']
optional_fields = ['cid']

In [26]:
def process_passport_input(inp):
    return tz.pipe(
        inp,
        lambda s: s.split('\n\n'),
        tz.map(lambda s: s.replace('\n', ' ')),
        tz.map(lambda s: s.split()),
        tz.map(lambda lst: {e.split(':')[0]: e.split(':')[1] for e in lst})
    )

def is_valid_passport(passport):
    return all(field in passport for field in required_fields)

In [27]:
def solve_4a(inp):
    return tz.pipe(
        inp, 
        tz.reduce(lambda a, x: a + '\n' + x),
        process_passport_input,
        tz.filter(is_valid_passport),
        tz.count
    )

In [28]:
solve_sample(solve_4a, sample_4), solve(solve_4a, 4)

(2, 170)

In [29]:
hcl_re = re.compile('#([a-f]|[0-9]){6}$')

def is_valid_passport_b(passport):
    if not is_valid_passport(passport):
        return False

    try:
        assert 1920 <= int(passport['byr']) <= 2002, "invalid byr"
    except:
        return False

    try:
        assert 2010 <= int(passport['iyr']) <= 2020, "invalid iyr"
    except:
        return False

    try:
        assert 2020 <= int(passport['eyr']) <= 2030, "invalid eyr"
    except:
        return False  

    try:
        hgt_unit = passport['hgt'][-2:]
        hgt_value = int(passport['hgt'][:-2])
        assert hgt_unit in {'cm', 'in'}
        if hgt_unit == 'cm':
            assert 150 <= hgt_value <= 193
        else:
            assert 59 <= hgt_value <= 76
    except:
        return False

    try: 
        assert hcl_re.search(passport['hcl'])
    except: 
        return False

    try:
        assert passport['ecl'] in {'amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth'}
    except:
        return False

    try:
        assert len(passport['pid']) == 9
        pid = int(passport['pid'])

    except:
        return False

    return True

In [30]:
def solve_4b(inp):
    return tz.pipe(
        inp, 
        tz.reduce(lambda a, x: a + '\n' + x),
        process_passport_input,
        tz.filter(is_valid_passport_b),
        tz.count
    )

In [31]:
solve_sample(solve_4b, sample_4), solve(solve_4b, 4)

(2, 103)

# day 5

In [32]:
sample_5 = """FBFBBFFRLR"""

In [33]:
def convert_binary(s, zero, one):
    return int(s.replace(zero, '0').replace(one, '1'), 2)

def row_number(boarding_pass):
    return convert_binary(boarding_pass[:7], 'F', 'B')

def column_number(boarding_pass):
    return convert_binary(boarding_pass[7:], 'L', 'R')

def decode_boarding_pass(boarding_pass):
    return row_number(boarding_pass), column_number(boarding_pass)

def convert_to_id(seat):
    row, column = seat
    return 8*row + column

In [34]:
def solve_5a(inp):
    return tz.pipe(
        inp,
        tz.map(decode_boarding_pass),
        tz.map(convert_to_id),
        max
    )

In [35]:
solve_sample(solve_5a, sample_5), solve(solve_5a, 5)

(357, 911)

In [36]:
def solve_5b(inp):
    return tz.pipe(
        inp,
        tz.map(decode_boarding_pass),
        tz.map(convert_to_id),
        sorted,
        tz.sliding_window(2),
        tz.reduce(lambda a, x: a if x[1] - x[0] == 1 else x[0] + 1)
    )

In [37]:
solve(solve_5b, 5)

629

# day 6

In [38]:
sample_6 = """abc

a
b
c

ab
ac

a
a
a
a

b"""

In [39]:
def solve_6a(inp):
    return tz.pipe(
            inp,
            tz.reduce(lambda a, x: a + '\n' + x),
            lambda s: s.split('\n\n'),
            tz.map(lambda s: s.replace('\n', '')),
            tz.map(set),
            tz.map(len),
            sum
    )

In [40]:
solve_sample(solve_6a, sample_6), solve(solve_6a, 6)

(11, 6625)

In [41]:
def solve_6b(inp):
    return tz.pipe(
            inp,
            tz.reduce(lambda a, x: a + '\n' + x),
            lambda s: s.split('\n\n'),
            tz.map(lambda s: s.split('\n')),
            tz.map(tz.reduce(lambda a, x: set(a).intersection(x))),
            tz.map(len),
            sum
    )

In [42]:
solve_sample(solve_6b, sample_6), solve(solve_6b, 6)

(6, 3360)

# day 7

In [43]:
sample_7 = """light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags."""

In [44]:
bag_count = re.compile("(\w+\s\w+\s\w+)\scontain\s((\d+\s\w+\s\w+\s\w+)[,.]\s*)*")

In [45]:


def parse_contains(contains):
    bags_raw = contains.split(',')
    if bags_raw == ['no other bags.']:
        return []

    else:
        return tz.pipe(
            bags_raw, 
            tz.map(lambda s: '-'.join(s.strip().split(" ")[1:3])), 
            list
        )

def parse_line(line):
    bag, contains = line.split(" contain ")
    return '-'.join(bag.split(" ")[:2]), parse_contains(contains)

def reverse_dict_list(d):
    reversed_d = collections.defaultdict(list)
    for key, values in d.items():
        for value in values:
            reversed_d[value].append(key)

    return dict(reversed_d)

@tz.curry
def bfs(start, graph):
    def _bfs(visited, queue, graph):
        if queue:
            node = queue.pop(0)
            return _bfs(visited.union([node]), queue + graph.get(node, []), graph)

        else:
            return visited

    return _bfs(set(), [start], graph)

In [46]:
def solve_7a(inp): 
    return tz.pipe(
        inp,
        tz.map(parse_line),
        dict,
        reverse_dict_list,
        bfs('shiny-gold'),
        len
        ) - 1

solve_sample(solve_7a, sample_7), solve(solve_7a, 7)   

(4, 211)

In [47]:
# sorry not sorry for copy-pasta

def parse_contains_count(contains):
    bags_raw = contains.split(',')
    if bags_raw == ['no other bags.']:
        return []

    else:
        return tz.pipe(
            bags_raw, 
            tz.map(lambda s: ('-'.join(s.strip().split(" ")[1:3]), int(s.strip().split(" ")[0]))), 
            list
        )

def parse_line_count(line):
    bag, contains = line.split(" contain ")
    return '-'.join(bag.split(" ")[:2]), parse_contains_count(contains)

@tz.curry
def count_all_the_bags(start, graph):
    if next_nodes := graph[start]:

        return sum(cnt * (1+count_all_the_bags(node, graph)) for node, cnt in next_nodes)
    return 0

In [48]:
def solve_7b(inp):
    return tz.pipe(
            inp,
            tz.map(parse_line_count),
            dict, 
            count_all_the_bags('shiny-gold')
        )

solve_sample(solve_7b, sample_7), solve(solve_7b, 7)

(32, 12414)

# day 8

In [49]:
sample_8 = """nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6"""

In [50]:
s = "acc -10"
def parse_instruction(op_string):
    split = op_string.split(' ')
    op = split[0]
    arg = int(split[1])
    return op, arg

In [52]:
@tz.curry
def exec_instructions(tape):
    """
    Returns (value of accumulator, is end of tape bool)
    """
    def exec(val, idx, visited):
        if idx >= len(tape):
            return val, True
        if idx in visited:
            return val, False
        
        op, arg = tape[idx]
        new_visited = visited.union([idx])
        if op == 'acc':
            return exec(val+arg, idx+1, new_visited)
        if op == 'jmp':
            return exec(val, idx+arg, new_visited)
        return exec(val, idx+1, new_visited)

    return exec(0, 0, set())


In [53]:
def solve_8a(inp):
    return tz.pipe(inp,
        tz.map(parse_instruction),
        list,
        exec_instructions,
        tz.first)

solve_sample(solve_8a, sample_8), solve(solve_8a, 8)

(5, 1521)

In [54]:
def try_fix_tape(tape):
    for idx, (op, arg) in enumerate(tape):
        if op in {'jmp', 'nop'}:
            new_tape = tape[:]
            new_tape[idx] = 'nop' if op == 'jmp' else 'jmp', arg
            yield new_tape

In [55]:
def solve_8b(inp):
    return tz.pipe(inp,
        tz.map(parse_instruction),
        list,
        try_fix_tape,
        tz.map(exec_instructions),
        tz.filter(lambda res: res[1]),  # find result that finishes at end of tape
        tz.first, # get the first solution
        tz.first  # get value of accumulator
        )
        
solve_sample(solve_8b, sample_8), solve(solve_8b, 8)

(8, 1016)

# day 9

In [56]:
sample_9 = """35
20
15
25
47
40
62
55
65
95
102
117
150
182
127
219
299
277
309
576"""

In [69]:
def solve_9a(inp, preamble=25):
    return tz.pipe(inp,
        tz.map(int),
        tz.sliding_window(preamble+1),
        tz.filter(tz.complement(is_valid_xmas_sequence)),
        tz.first,
        tz.last
    )

solve_sample(solve_9a, sample_9, 5), solve(solve_9a, 9)

(127, 1492208709)

In [83]:
def is_valid_xmas_sequence(seq):
    *prev, target = seq
    soln = two_elems_sum_to_target(target, prev)
    return soln is not None

In [111]:
@tz.curry
def contiguous_sum(target, seq):
    """
    Return a contiguous subsequence that sums to target value
    """
    def recurse(start, end, current):
        if current == target:
            return seq[start:end]
        if current > target:
            return recurse(start+1, end, current-seq[start])
        if current < target:
            return recurse(start, end+1, current+seq[end])
    # could refactor to only use generator to save memory
    seq = list(seq)
    return recurse(0, 0, 0)

In [115]:
def solve_9b(inp, preamble=25):
    # need to use input twice
    inp = list(inp)
    target = solve_9a(inp, preamble)
    return tz.pipe(
        inp,
        tz.map(int),
        contiguous_sum(target),
        lambda seq: min(seq) + max(seq)
    )

solve_sample(solve_9b, sample_9, 5), solve(solve_9b, 9, 25)

(62, 238243506)

(62, 238243506)